In [ ]:
import skimage.restoration as sr
import numpy as np
import glob
# import h5py
import os
import scipy.io as scio
from skimage import exposure
from skimage.io import imsave, imread
from scipy.misc import imresize
from scipy.io import savemat
from scipy import ndimage, misc
import matplotlib.pyplot as plt
%matplotlib inline
import re
import cv2

from helper import *

import os
cwd = os.getcwd()

In [ ]:
def atoi(text) : 
    return int(text) if text.isdigit() else text

def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]

### Remove Images in Folder

In [ ]:
remove = False
if remove:
    nlmeans_image_folder_rem = cwd + '/datasets/OCTData/alldata/nl_means_denoised/Train/'
    remove_files(nlmeans_image_folder_rem, '.png')

### Root Folder

Removed folders in whole_data is os.walk will reference them

In [ ]:
root_path = ""
filenames = []
# NOTE: REMOVE SUBFOLDER FROM THIS FOLDER SO FILENAMES LENGTH IS CORRECT!!!
# for root, dirnames, filenames in os.walk("/home/sim/notebooks/relaynet_pytorch/datasets/OCTData/alldata/whole_raw_image/Train"):
for root, dirnames, filenames in os.walk("/Users/sim/Desktop/Imperial/Project/PreTrained/relaynet_pytorch/datasets-24-aug/OCTData/alldata/whole_raw_image"):
    filenames.sort(key = natural_keys)
    rootpath = root
print(len(filenames))

In [ ]:
# print(filenames)
filenames = filenames[1:] # making filenames without .DS_store
# print(filenames)

In [ ]:
def get_info(filenames, ext):
    images = []
    for filename in filenames :
        filepath = os.path.join(root,filename)
        if ext == '.npy':
            image = np.load(filepath)
            h,w = image.shape
            
            if h != 512 or w != 64:
#                 print(h,w) 
                amount = 512 - h
                id_full = np.full((amount, 64), 0)
                image = np.concatenate((image, id_full))
#                 print(image.shape)
        elif ext == '.JPG' or ext == '.tif':
            image = ndimage.imread(filepath, mode = "L")
        images.append(image)
    return images

In [ ]:
raw_images = get_info(filenames, '.tif')
print (len(raw_images))
# print(filenames)

### Compute Gradient Map
Source: https://handmap.github.io/gradients-and-edge-detection/

In [ ]:
# loading image
def find_contours(new_img):
    x_sz, y_sz = 9, 9

    
    # remove noise    


    img = new_img

    plt.figure(figsize=(x_sz,y_sz))
    plt.imshow(img, cmap="gray")
    plt.title('DeNoisedImage'), plt.xticks([]), plt.yticks([])
#     plt.show()
    
    # Compute the Laplacian of the image
    lap = cv2.Laplacian(img, cv2.CV_64F)
    lap = np.uint8(np.absolute(lap))

    plt.figure(figsize=(x_sz,y_sz))
    plt.imshow(lap, cmap="gray")
    plt.title('Laplacian'), plt.xticks([]), plt.yticks([])
    
    # Compute gradients along the X and Y axis, respectively
    sobelX = cv2.Sobel(lap, cv2.CV_64F, 1, 0)
    sobelY = cv2.Sobel(lap, cv2.CV_64F, 0, 1)
    
    # The sobelX and sobelY images are now of the floating
    # point data type -- we need to take care when converting
    # back to an 8-bit unsigned integer that we do not miss
    # any images due to clipping values outside the range
    # of [0, 255]. First, we take the absolute value of the
    # graident magnitude images, THEN we convert them back
    # to 8-bit unsigned integers
    sobelX = np.uint8(np.absolute(sobelX))
    sobelY = np.uint8(np.absolute(sobelY))
    
    # We can combine our Sobel gradient images using our
    # bitwise OR
    sobelCombined = cv2.bitwise_or(sobelX, sobelY)
    
    # Show our Sobel images
    plt.figure(figsize=(x_sz,y_sz))
    plt.imshow(sobelX, cmap='gray')
    plt.title('Sobelx'), plt.xticks([]), plt.yticks([])
    
    plt.figure(figsize=(x_sz,y_sz))
    plt.imshow(sobelY, cmap='gray')
    plt.title('Sobely'), plt.xticks([]), plt.yticks([])
    
    plt.figure(figsize=(x_sz,y_sz))
    plt.imshow(sobelCombined, cmap='gray')
    plt.title('sobelCombined'), plt.xticks([]), plt.yticks([])
    
    return sobelCombined

#     # convolute with proper kernels
#     laplacian = cv2.Laplacian(img,cv2.CV_64F)
#     laplacian = np.uint8(np.absolute(laplacian))
    
#     sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)  # x
#     sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)  # y

#     plt.subplot(2,2,1),plt.imshow(img,cmap = 'gray')
#     plt.title('Original'), plt.xticks([]), plt.yticks([])
#     plt.subplot(2,2,2),plt.imshow(laplacian, cmap = 'gray')
#     plt.title('Laplacian'), plt.xticks([]), plt.yticks([])
#     plt.subplot(2,2,3),plt.imshow(sobelx, cmap = 'gray')
#     plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
#     plt.subplot(2,2,4),plt.imshow(sobely, cmap = 'gray')
#     plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])
#     plt.show()

In [ ]:
image = '/Users/sim/Desktop/Imperial/Project/PreTrained/relaynet_pytorch/datasets-24-aug/OCTData/alldata/whole_raw_image/con_H1_N01848_LA_1_272.tif'
# Load the image
img = cv2.imread(image)
returned_image = compute_laplacian_gm(img)

In [ ]:
gradient_map_images = []
for count, image in enumerate(raw_images):
    if count < 1:
        plt.imshow(image)
        compute_laplacian_gm(image)
        print(count)
        gradient_map_images.append(compute_laplacian_gm(image))

In [ ]:
print(len(gradient_map_images))

In [ ]:
directory = '/Users/sim/Desktop/Imperial/Project/PreTrained/relaynet_pytorch/datasets-24-aug/OCTData/alldata/'
for item in range(len(raw_images)):
    name = filenames[item]
    
    gm_file = directory + 'gradient_map/'+ name
    
    # Saving files
    imsave(gm_file, gradient_map_images[item])